Time series

In [ ]:
# Import the dataset
data = pd.read_csv('')

data.head()

In [ ]:
# Plot the data 
plt.plot(data[''], 'o-')
plt.xlabel('')
plt.ylabel('')
plt.title('Time series plot')
plt.grid()
plt.show()

In [ ]:
# Runs test for randomness
_, pval_runs = runstest_1samp(data[''], correction=False)
print('Runs test p-value = {:.3f}'.format(pval_runs))

In [ ]:
# Plot acf and pacf 

fig, ax = plt.subplots(2, 1)
sgt.plot_acf(data[''], lags = int(len(data)/3), zero=False, ax=ax[0])
fig.subplots_adjust(hspace=0.5)
sgt.plot_pacf(data[''], lags = int(len(data)/3), zero=False, ax=ax[1], method = 'ywm')
plt.show()

In [ ]:
#calculate the lag k from data
k=1
data['lagk'] = data[''].shift(k)

#create scatterplot with regression line using seaborn and set axis labels
sns.regplot(x=data['lagk'], y=data[''], ci=None, line_kws={'color':'red', 'ls':'--'})
plt.title('Scatter plot of X(t-k) vs X(t)')
plt.xlabel('X(t-k)')
plt.ylabel('X(t)')
plt.title('Scatter plot of X(t-k) vs X(t)')
plt.grid()


In [ ]:
#calculate the difference between the data and the lagk
data['diffk'] = data[''] - data['lagk']

plt.plot(data['diffk'], 'o-')
plt.xlabel('Index')
plt.ylabel('DIFF k')
plt.title('Time series plot of DIFF k')
plt.grid()
plt.show()

In [ ]:
n = len(data['diffk'][1:])

#autocorrelation function
[acf_values, lbq, _] = acf(data['diffk'][1:], nlags = int(np.sqrt(n)), qstat=True, fft = False)

#Bartlett's test at lag k
alpha = 0.05
lag_test = k
rk = acf_values[lag_test]
z_alpha2 = stats.norm.ppf(1-alpha/2)
print('Test statistic rk = %f' % rk)
print('Rejection region starts at %f' % (z_alpha2/np.sqrt(n)))

In [ ]:
#calculate a regression model with constant and lagk
x = data['lagk'][1:]
x = sm.add_constant(data['lagk'][1:]) # this command is used to consider a constant to the model, is equivalent to create and add a column of ones
y = data[''][1:]
model = sm.OLS(y, x).fit()

qda.summary(model)

In [ ]:
# without constant term
x = data['lagk'][1:]
y = data[''][1:]
model = sm.OLS(y, x).fit()
qda.summary(model)

In [ ]:
#assumptions on model residuals
fig, axs = plt.subplots(2, 2)
fig.suptitle('Residual Plots')
stats.probplot(model.resid, dist="norm", plot=axs[0,0])
axs[0,0].set_title('Normal probability plot')
axs[0,1].scatter(model.fittedvalues, model.resid)
axs[0,1].set_title('Versus Fits')
fig.subplots_adjust(hspace=0.5)
axs[1,0].hist(model.resid)
axs[1,0].set_title('Histogram')
axs[1,1].plot(np.arange(1, len(model.resid)+1), model.resid, 'o-')
_, pval_SW_res = stats.shapiro(model.resid)
print('Shapiro-Wilk test p-value on the residuals = %.3f' % pval_SW_res)

In [ ]:
#assumptions on model residuals
_, pval_runs_res = runstest_1samp(model.resid, correction=False)
print('Runs test p-value on the residuals = {:.3f}'.format(pval_runs_res))
fig, ax = plt.subplots(2, 1)
sgt.plot_acf(model.resid, lags = int(len(data)/3), zero=False, ax=ax[0])
fig.subplots_adjust(hspace=0.5)
sgt.plot_pacf(model.resid, lags = int(len(data)/3), zero=False, ax=ax[1], 
            method = 'ywm')
plt.show()

In [ ]:
# to confront original values vs fitted
plt.plot(data['EXE1'], 'o-', label='Original data')
plt.xlabel('Index') 
plt.ylabel('EXE1')
plt.plot(model.fittedvalues, 's--', color='red', label='Fitted values', alpha=0.5)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# fit model ARIMA with constant term
x = data['']
model = qda.ARIMA(x, order=(0,1,1), add_constant = True) 
# order=(p,d,q) (auto reg, diff, mov aver)

qda.ARIMAsummary(model)

In [ ]:
#extract the residuals
residuals = model.resid[1:]

# Perform the Shapiro-Wilk test
_, pval_SW = stats.shapiro(residuals)
print('Shapiro-Wilk test p-value = %.3f' % pval_SW)

# Plot the qqplot
stats.probplot(residuals, dist="norm", plot=plt)
plt.show()